In [1]:
# Import the necessary library
import zipfile

# Define the path to your zip file
zip_file_path = "/content/drive/MyDrive/Colab Notebooks/datasets/DATA.zip"  # Replace with the actual path to your zip file

# Define the directory where you want to extract the contents
extracted_dir_path = "/content"  # You can change this to your desired directory

# Create the target directory if it doesn't exist
import os
os.makedirs(extracted_dir_path, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir_path)

# List the contents of the extracted directory
extracted_contents = os.listdir(extracted_dir_path)
print("Extracted contents:", extracted_contents)


Extracted contents: ['.config', 'drive', 'DATA', 'sample_data']


In [14]:
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [ ]:
IMAGE_SIZE = [640, 640]


train_path = '/content/DATA/train'
valid_path = '/content/DATA/valid'
test_path  = '/content/DATA/test'

# useful for getting number of classes
folders = glob('/content/DATA/train/*')

len(folders)


# Model_making

In [27]:
# Define the EfficientNetB1 model with pre-trained weights, excluding the top layer
base_model = EfficientNetB1(weights='imagenet', include_top=False, input_shape=(*IMAGE_SIZE, 3))

# Fine-tune all layers
for layer in base_model.layers:
    layer.trainable = True


# Additional convolutional layers
x = base_model.output

x = Conv2D(64, (3, 3), activation='relu')(x)  # Add a new convolutional layer
x = MaxPooling2D(pool_size=(2, 2))(x)        # Add max pooling

x = Conv2D(128, (3, 3), activation='relu')(x)  # Add another convolutional layer
x = MaxPooling2D(pool_size=(2, 2))(x)         # Add max pooling

# Flatten the output and add fully connected layers
x = Flatten()(x)
x = Dense(256, activation='relu')(x)           # Add a dense layer
x = Dropout(0.2)(x)                           # Add dropout for regularization
prediction = Dense(len(folders), activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=prediction)

# View the structure of the model
model.summary()

# Compile the model with the desired loss, optimizer, and metrics
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 640, 640, 3)]        0         []                            
                                                                                                  
 rescaling_4 (Rescaling)     (None, 640, 640, 3)          0         ['input_3[0][0]']             
                                                                                                  
 normalization_2 (Normaliza  (None, 640, 640, 3)          7         ['rescaling_4[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_5 (Rescaling)     (None, 640, 640, 3)          0         ['normalization_2[0][0]'

# Data Processing

In [38]:
# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

# Define the data augmentation parameters
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,                # Shear transformation
    zoom_range=0.2,                 # Zoom transformation
    horizontal_flip=True,           # Horizontal flip
    fill_mode='nearest'             # Fill mode for new pixels
)
valid_datagen = ImageDataGenerator(rescale = 1./255)

# Specify the path to your dataset
train_data_dir = '/content/DATA/train'
valid_data_dir = '/content/DATA/valid'

# Define the number of images you want to select
num_images_to_select_train = 200
num_images_to_select_valid =50 # Change this to your desired number

# Get a list of subdirectories (class folders) in the train and valid directories
train_classes = os.listdir(train_data_dir)
valid_classes = os.listdir(valid_data_dir)

# Create a dictionary to store the selected images
selected_train_images = {}
selected_valid_images = {}

# Loop through each class and select a subset of images
for class_name in train_classes:
    class_dir = os.path.join(train_data_dir, class_name)
    image_files = os.listdir(class_dir)
    selected_train_images[class_name] = image_files[:num_images_to_select_train]

for class_name in valid_classes:
    class_dir = os.path.join(valid_data_dir, class_name)
    image_files = os.listdir(class_dir)
    selected_valid_images[class_name] = image_files[:num_images_to_select_valid]

# Create the data generators with the selected images
train_set = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(640, 640),
    batch_size=4,
    class_mode='categorical',
    classes=list(selected_train_images.keys()),
    subset='training',  # Use only selected images
    shuffle=True  # You can change this as needed
)

valid_set = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(640, 640),
    batch_size=4,
    class_mode='categorical',
    classes=list(selected_valid_images.keys()),
    subset='training',  # Use only selected images
    shuffle=True  # You can change this as needed
)

Found 2575 images belonging to 7 classes.
Found 795 images belonging to 7 classes.


# Training

In [36]:
# Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    "model_weights.h5",  # Path to save the best weights
    monitor="val_loss",   # Metric to monitor (validation loss)
    save_best_only=True,  # Save only the best model weights
    mode="min",           # Minimize the monitored metric (val_loss)
    verbose=1             # Display messages
)

# Fit the model
batch_size =2
history = model.fit(
    train_set,
    validation_data=valid_set,
    epochs=50,
    steps_per_epoch=len(train_set) // batch_size,
    validation_steps=len(valid_set) // batch_size,
    callbacks=[checkpoint]
)


Epoch 1/50


ResourceExhaustedError: ignored

# plots

In [ ]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')



# accuracies
plt.plot(history.history['acc'], label='train acc')
plt.plot(history.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

# Save Model

In [ ]:
import tensorflow as tf

from keras.models import load_model

model.save('model.h5')
